In [38]:
import os
import glob

import pandas as pd
from python_pdb.parsers import parse_pdb
from python_pdb.formats.residue import THREE_TO_ONE_CODE

In [2]:
DATA_DIR = '/project/koohylab/bmcmaste/projects/tcr-loop-comparison/data/apo-holo-mhc-class-I_refined'

In [5]:
df = pd.DataFrame({'path': glob.glob('**/*.pdb', root_dir=DATA_DIR, recursive=True)})
df

,path
0,YSGSPE-HISR-ALSGFNNAGNMLT-SGHAT-FQNNGV-ASSLGGA...
1,YSGSPE-HISR-ALSGFNNAGNMLT-SGHAT-FQNNGV-ASSLGGA...
2,NIATNDY-GYKTK-LVGEILDNFNKFY-MDHEN-SYDVKM-ASSQR...
3,NIATNDY-GYKTK-LVGEILDNFNKFY-MDHEN-SYDVKM-ASSQR...
4,DRGSQS-IYSNGD-GTYNQGGKLI-MNHEY-SMNVEV-ASSGASHE...
...,...
70,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGT...
71,NSAFDY-ILSVSNK-AASASFGDNSKLI-MSHET-SYDVDS-ASSL...
72,NSAFDY-ILSVSNK-AASASFGDNSKLI-MSHET-SYDVDS-ASSL...
73,DSAIYN-IQSSQRE-AQLNQAGTALI-MNHEY-SVGAGI-ASSYGT...


In [27]:
df['tcr_group'] = df['path'].map(lambda path: path.split('/')[0])
df['state'] = df['path'].map(lambda path: path.split('_')[-1].split('.')[0])

chains = df['path'].map(lambda path: path.split('_')[-2])
df[['alpha_chain', 'beta_chain', 'antigen_chain', 'mhc_chain']] = chains.apply(list).apply(pd.Series)

df

,path,tcr_group,alpha_chain,beta_chain,antigen_chain,mhc_chain,state
0,YSGSPE-HISR-ALSGFNNAGNMLT-SGHAT-FQNNGV-ASSLGGA...,YSGSPE-HISR-ALSGFNNAGNMLT-SGHAT-FQNNGV-ASSLGGA...,D,E,NaN,NaN,apo
1,YSGSPE-HISR-ALSGFNNAGNMLT-SGHAT-FQNNGV-ASSLGGA...,YSGSPE-HISR-ALSGFNNAGNMLT-SGHAT-FQNNGV-ASSLGGA...,D,E,C,A,holo
2,NIATNDY-GYKTK-LVGEILDNFNKFY-MDHEN-SYDVKM-ASSQR...,NIATNDY-GYKTK-LVGEILDNFNKFY-MDHEN-SYDVKM-ASSQR...,A,B,NaN,NaN,apo
3,NIATNDY-GYKTK-LVGEILDNFNKFY-MDHEN-SYDVKM-ASSQR...,NIATNDY-GYKTK-LVGEILDNFNKFY-MDHEN-SYDVKM-ASSQR...,A,B,P,H,holo
4,DRGSQS-IYSNGD-GTYNQGGKLI-MNHEY-SMNVEV-ASSGASHE...,DRGSQS-IYSNGD-GTYNQGGKLI-MNHEY-SMNVEV-ASSGASHEQY,A,B,NaN,NaN,apo
...,...,...,...,...,...,...,...
70,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGT...,DRGSQS-IYSNGD-AVNFGGGKLI-MRHNA-SNTAGT-ASSLSFGTEAF,D,E,C,A,holo
71,NSAFDY-ILSVSNK-AASASFGDNSKLI-MSHET-SYDVDS-ASSL...,NSAFDY-ILSVSNK-AASASFGDNSKLI-MSHET-SYDVDS-ASSL...,E,F,P,A,holo
72,NSAFDY-ILSVSNK-AASASFGDNSKLI-MSHET-SYDVDS-ASSL...,NSAFDY-ILSVSNK-AASASFGDNSKLI-MSHET-SYDVDS-ASSL...,A,B,NaN,NaN,apo
73,DSAIYN-IQSSQRE-AQLNQAGTALI-MNHEY-SVGAGI-ASSYGT...,DSAIYN-IQSSQRE-AQLNQAGTALI-MNHEY-SVGAGI-ASSYGT...,A,B,NaN,NaN,apo


In [32]:
A1 = range(1, 14 + 1)
B1 = range(18, 28 + 1)
C1 = range(31, 38 + 1)
A2 = range(1001, 1014 + 1)
B2 = range(1018, 1028 + 1)
C2 = range(1031, 1038 + 1)
D2 = range(1042, 1049 + 1)

In [40]:
def get_sequence(df):
    new_df = df.drop_duplicates(['chain_id', 'residue_seq_id', 'residue_insert_code'])
    return ''.join(new_df['residue_name'].map(lambda tlc: THREE_TO_ONE_CODE[tlc]).to_list())

def align(mobile: pd.DataFrame, target: pd.DataFrame):
    mobile_sequence = get_sequence(mobile)
#     target_sequence = get_sequence(target)
    
    print(mobile_sequence)
    
    

for (group_name, group_df) in df.groupby('tcr_group'):
    first_entry = group_df.query("state == 'holo'").iloc[0]
    
    with open(os.path.join(DATA_DIR, first_entry['path']), 'r') as fh:
        reference_structure = parse_pdb(fh.read()).to_pandas()
    
    reference_mhc = reference_structure.query("chain_id == @first_entry.mhc_chain")
    
    
    # Rough alignment
    # Dynamic programming to find structurally similar residues
    # 
    align(reference_mhc, None)
    
    break

GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRMEPRAPWIEQEGPEYWDGETRKVKAHSQTHRVDLGTLRGYYNQSEAGSHTVQRMYGCDVGSDWRFLRGYHQYAYDGKDYIALKEDLRSWTAADMAAQTTKHKWEAAHVAEQLRAYLEGTCVEWLRRYLENGKETLQRTDAPKTHMTHHAVSDHEATLRCWALSFYPAEITLTWQRDGEDQTQDTELVETRPAGDGTFQKWAAVVVPSGQEQRYTCHVQHEGLPKPLTLRWE
